In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

#### sklearn.model_selection.cross_val_score
* sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
* cv → 폴드 갯수
* fit_params → 교차 검증으로 나온 최적 모형을 학습까지 시켜줌   
dict
* Markdown에서 줄바꿈 → 마지막 줄에서 세칸 이상 띄어쓰기 or Tab, 줄바꿈

In [3]:
iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scroes = cross_val_score(dt_clf, data, label, scoring = "accuracy", cv = 3)
print("교차 검증별 정확도 : ", np.round(scroes, 4))
print("평균 검증 정확도 : ", np.round(np.mean(scroes), 4))

# KFold 에서 사용해야 하는 for문 없이 바로 결과 나옴

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


#### GridSerchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에
* class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [14]:
# 데이터를 로딩하고 학습데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size = 0.2, random_state = 121)

dtree = DecisionTreeClassifier()

# parameter 들을 dictionary 형태로 설정
parameters = {"max_depth" : [1,2,3], "min_samples_split":[2,3]}

# 아래의 GridSearchCV 에서 사용

In [26]:
import pandas as pd
from sklearn.metrics import accuracy_score

# param_grid 의 하이퍼 파라미터들을 3개의 train, test_set_fold 로 
# 나누어서 테스트 수행 설정.
# refit = True 가 default 임. True 이면 가장 좋은 파라미터 설정으로 재학습 시킴.
grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv = 3, refit = True)

# 붓꽃 Train 데이터로 param_grid 의 하이퍼 파라미터들을 순차적으로 학습/평가.
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame 으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[["params", "mean_test_score", "rank_test_score",
            "split0_test_score", "split1_test_score", "split2_test_score"]]
print(f"GridSearchCV 최적 파라미터 : {grid_dtree.best_params_}")
print(f"GridSearchCV 최고 정확도 : {grid_dtree.best_score_ : .4f}")

GridSearchCV 최적 파라미터 : {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도 :  0.9750


In [27]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print(f"테스트 데이터 세트 정확도 {accuracy_score(y_test, pred):.4f}")

테스트 데이터 세트 정확도 0.9667


In [28]:
X_test

array([[7. , 3.2, 4.7, 1.4],
       [6.2, 3.4, 5.4, 2.3],
       [6. , 3.4, 4.5, 1.6],
       [4.6, 3.4, 1.4, 0.3],
       [5.2, 4.1, 1.5, 0.1],
       [5.8, 2.6, 4. , 1.2],
       [5.6, 2.9, 3.6, 1.3],
       [5.7, 2.6, 3.5, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [6.9, 3.1, 5.1, 2.3],
       [5.9, 3.2, 4.8, 1.8],
       [5.5, 2.3, 4. , 1.3],
       [6.6, 2.9, 4.6, 1.3],
       [4.6, 3.2, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [6.3, 3.4, 5.6, 2.4],
       [6. , 2.2, 4. , 1. ],
       [5. , 3. , 1.6, 0.2],
       [6.4, 3.2, 5.3, 2.3],
       [5.4, 3.9, 1.7, 0.4],
       [6.4, 2.7, 5.3, 1.9],
       [6.3, 2.5, 5. , 1.9],
       [6. , 2.9, 4.5, 1.5],
       [5.1, 2.5, 3. , 1.1],
       [5.5, 2.6, 4.4, 1.2],
       [5.5, 2.5, 4. , 1.3],
       [5. , 3.3, 1.4, 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [7.9, 3.8, 6.4, 2. ],
       [7.1, 3. , 5.9, 2.1]])

In [29]:
y_test

array([1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 2, 1, 0, 2, 0, 2, 2,
       1, 1, 1, 1, 0, 0, 2, 2])

In [35]:
estimator.predict([[7. , 3.2, 4.7, 1.4]])

array([1])

In [36]:
estimator.predict_proba([[7. , 3.2, 4.7, 1.4]]) # 확률로 보여줌.
                                            # 아마 가장 높은 확률 또는 90% 이상

array([[0.        , 0.97142857, 0.02857143]])